In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [2]:
def merge_external_data(df, symbols, allow_exact_matches=False):
    """
    Merge other stock market data (external data) into the main DataFrame using merge_asof.
    
    df: main DataFrame with a sorted 'Date' column
    symbols: list of external CSV filenames
    allow_exact_matches: whether to allow exact date matches in merge_asof (US stock data will cause data leakage, so we need to shift them for one day)
    """
    for sym in symbols:
        file_path = f"./data/{sym}.csv"

        # remove the old one
        if sym in df.columns:
            df.drop(columns=[sym], inplace=True)

        try:
            df_tmp = pd.read_csv(file_path)

            # Convert Date column to datetime
            df_tmp['Date'] = pd.to_datetime(df_tmp['Date'])

            # Target column to use from external data
            target_col = 'rate' # rate for price change

            if target_col not in df_tmp.columns:
                print(f"Skip {sym}: column '{target_col}' not found")
                continue

            # Clean numeric values (remove thousand separators and transform to float)
            if df_tmp[target_col].dtype == 'object':
                df_tmp[target_col] = df_tmp[target_col].str.replace(',', '', regex=False).astype(float)
            else:
                df_tmp[target_col] = df_tmp[target_col].astype(float)

            # Sort external data by Date 
            df_tmp.sort_values(by='Date', inplace=True)

            # Keep only Date and target column, and rename to symbol name
            external_data = df_tmp[['Date', target_col]].rename(columns={target_col: sym})

            # Merge with main DataFrame using backward asof join
            # This takes the most recent available value on or before the main date
            # allow_exact_matches:
            #   True -> allow matching data from the same day if available; 
            #   False -> do NOT use same-day data even if it exists; always use the most recent strictly earlier day
            df = pd.merge_asof(
                df,
                external_data,
                on='Date',
                direction='backward',
                allow_exact_matches=allow_exact_matches
            )

        except Exception as e:
            print(f"Error processing {sym}: {e}")

    return df

In [3]:
# Add information about other stocks as the feature.
# We cannot directly merge the CSV files because different stocks may have different trading dates.
# Using merge_asof allows us to align dates and take the most recent available data.

filename = 'data/2330.csv'
df = pd.read_csv(filename)

# Convert Date column to datetime and sort 
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
df.sort_values(by='Date', inplace=True) 

# Merge U.S. stock market data
stk_us = ['DJI', 'NASDAQ', 'SOX', 'SPX', 'ADR']
df = merge_external_data(df, stk_us, allow_exact_matches=False)

# Merge Taiwan market data and the exchange rate of the NTD to the USD
stk_tw = ['tw', 'TWD']
df = merge_external_data(df, stk_tw, allow_exact_matches=True)

df.head()

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transaction,rate,...,MACD_signal,MACD_hist,ADX,DJI,NASDAQ,SOX,SPX,ADR,tw,TWD
0,2010-01-04,39511138,2557720928,65.0,65.0,64.0,64.9,0.4,8255,NaN,...,NaN,NaN,NaN,-1.141962,-0.965841,-0.409537,-1.004960,1.418412,0.040000,0.958466
1,2010-01-05,38394084,2464115096,65.0,65.1,63.9,64.5,-0.4,9205,-0.616333,...,NaN,NaN,NaN,1.495104,1.730605,1.714311,1.604342,1.223796,0.043251,-0.634921
2,2010-01-06,52734385,3390698544,64.5,64.9,63.7,64.9,0.4,12597,0.620155,...,NaN,NaN,NaN,-0.112816,0.012564,0.092886,0.311568,-0.431779,1.415349,0.000000
3,2010-01-07,53294614,3437221996,64.9,65.0,64.2,64.2,-0.7,11195,-1.078582,...,NaN,NaN,NaN,0.015703,-0.330049,-0.021840,0.054552,-0.346954,-1.083143,0.000000
4,2010-01-08,48047497,3068341466,63.5,64.3,63.5,64.0,-0.2,9804,-0.311526,...,NaN,NaN,NaN,0.313804,-0.045198,-1.102803,0.400120,-3.307171,0.527835,0.000000


In [4]:
# Convert all columns except 'Date' to numeric ('1000' to 1000)
# This ensures all feature columns are numeric for modeling
df.loc[:, df.columns != "Date"] = df.loc[:, df.columns != "Date"].apply(pd.to_numeric, errors="coerce")

# Fill missing values
df.ffill(inplace=True)
df.bfill(inplace=True)

df.to_csv("data/df.csv", index=False)
df.head(5)

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transaction,rate,...,MACD_signal,MACD_hist,ADX,DJI,NASDAQ,SOX,SPX,ADR,tw,TWD
0,2010-01-04,39511138,2557720928,65.0,65.0,64.0,64.9,0.4,8255,-0.616333,...,-1.786677,0.378051,55.4694,-1.141962,-0.965841,-0.409537,-1.004960,1.418412,0.040000,0.958466
1,2010-01-05,38394084,2464115096,65.0,65.1,63.9,64.5,-0.4,9205,-0.616333,...,-1.786677,0.378051,55.4694,1.495104,1.730605,1.714311,1.604342,1.223796,0.043251,-0.634921
2,2010-01-06,52734385,3390698544,64.5,64.9,63.7,64.9,0.4,12597,0.620155,...,-1.786677,0.378051,55.4694,-0.112816,0.012564,0.092886,0.311568,-0.431779,1.415349,0.000000
3,2010-01-07,53294614,3437221996,64.9,65.0,64.2,64.2,-0.7,11195,-1.078582,...,-1.786677,0.378051,55.4694,0.015703,-0.330049,-0.021840,0.054552,-0.346954,-1.083143,0.000000
4,2010-01-08,48047497,3068341466,63.5,64.3,63.5,64.0,-0.2,9804,-0.311526,...,-1.786677,0.378051,55.4694,0.313804,-0.045198,-1.102803,0.400120,-3.307171,0.527835,0.000000


In [ ]:
df = pd.read_csv("data/df.csv")

#Define split dates for train/test
split_date_start = pd.to_datetime("2009/1/1")
split_date_mid = pd.to_datetime("2025/1/1")
split_date_end   = pd.to_datetime("2026/1/1")

# Ensure Date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# split into training set and testing set
df_train = df[(df["Date"] >= split_date_start) & (df["Date"] < split_date_mid)].copy()
df_test = df[(df["Date"] >= split_date_mid )& (df["Date"] < split_date_end)].copy()

# find feature columns
exclude_cols = ["Movement", "Date"]
feature_cols = [c for c in df.columns if c not in exclude_cols]

scaler = RobustScaler()

# Fit scaler on training set and transform train and test set
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

df_train.to_csv("data/train.csv", index=False)
df_test.to_csv("data/test.csv", index=False)
